## 문제 6

**Kaggle 형** train_prob.csv로 문제 target을 예측하는 모델을 만들고, 

test_prob.csv에 대한 target 예측하여 다음과 같은 형식의 answer6.csv를 만들어라.

id, target

0, 6.9

5, 7.8

...


**평가지표**

$RMSE(Y, \hat{Y}) = \sqrt{\frac{1}{n}\sum^{n}_{i=1}(y_i-\hat{y_i})^2}$


In [1]:
import warnings
with warnings.catch_warnings(): 
    warnings.simplefilter("ignore")
    import pandas as pd
    import numpy as np
    import sklearn
    import scipy
    import statsmodels
    import mlxtend
    import sys

print(sys.version)
for i in [pd, np, sklearn, scipy, mlxtend, statsmodels]:
    print(i.__name__, i.__version__)

3.7.4 (default, Aug 13 2019, 20:35:49) 
[GCC 7.3.0]
pandas 1.3.5
numpy 1.21.5
sklearn 0.21.3
scipy 1.5.2
mlxtend 0.15.0.0
statsmodels 0.13.5


In [2]:
df_train = pd.read_csv("train_prob.csv", index_col="id")
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 240000 entries, 267387 to 415856
Data columns (total 25 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   cat0    240000 non-null  object 
 1   cat1    240000 non-null  object 
 2   cat2    240000 non-null  object 
 3   cat3    240000 non-null  object 
 4   cat4    240000 non-null  object 
 5   cat5    240000 non-null  object 
 6   cat6    240000 non-null  object 
 7   cat7    240000 non-null  object 
 8   cat8    240000 non-null  object 
 9   cat9    240000 non-null  object 
 10  cont0   240000 non-null  float64
 11  cont1   240000 non-null  float64
 12  cont2   240000 non-null  float64
 13  cont3   240000 non-null  float64
 14  cont4   240000 non-null  float64
 15  cont5   240000 non-null  float64
 16  cont6   240000 non-null  float64
 17  cont7   240000 non-null  float64
 18  cont8   240000 non-null  float64
 19  cont9   240000 non-null  float64
 20  cont10  240000 non-null  float64
 21  cont1

In [3]:
df_train["target A"] = df_train["target"] <= 7.45
df_train.head()

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target,target A
id,,,,,,,,,,,,,,,,,,,,,
267387,A,B,B,C,B,B,A,E,G,G,...,0.69220,0.30912,0.47134,0.53290,0.97139,0.59414,0.36856,0.42951,8.13741,False
410037,A,A,A,A,B,B,A,E,C,F,...,0.97199,0.62596,0.40193,0.49181,0.56479,0.39386,0.36247,0.71541,7.40427,True
139373,A,A,B,A,B,B,A,E,C,J,...,0.41310,0.27010,0.40112,0.53052,0.35635,0.33341,0.33354,0.82542,7.98022,False
113765,A,A,A,C,B,B,A,E,C,G,...,0.42301,0.28331,0.46425,0.38302,0.38054,0.68827,0.34895,0.73375,7.26085,True
179915,A,B,A,C,B,B,A,E,E,L,...,0.37466,0.46316,0.75578,0.55676,0.31308,0.43157,0.32302,0.18028,7.26448,True


In [4]:
# 처리할 내용을 튜플과 리스트로 만듭니다.
repl_list = [
    ('cat3', {'B': 'C'}, [83634, 147361, 9005]),
    ('cat4', {'A': 'B', 'D': 'B'}, [239397, 603]), 
    ('cat6', {'D': 'A', 'E': 'B', 'G': 'C', 'H': 'B', 'I': 'A'}, [234203, 5145, 652]),
    ('cat7', {'A': 'B', 'C': 'B', 'F': 'D', 'I': 'B'}, [4606, 19784, 214027, 1583]),
    ('cat8', {'B': 'G', 'F': 'E'}, [30338, 96743, 2953, 76085, 33881]),
    ('cat9', {'C': 'H', 'D': 'B', 'E': 'L'}, [10678, 2846, 85944, 8320, 19987, 40070, 5501, 16743, 33793, 7819, 3331, 4968])
]

In [5]:
# 반복문을 통해 처리합니다.
for c, d, cnt in repl_list:
    # 위에서 설계한 로직을 적용합니다.
    # 처리한 값을 저장합니다.
    df_train[c] = df_train[c].replace(d)
    # 카운트가 다른 게 있는지 확인 합니다.
    if (df_train[c].value_counts().sort_index() != cnt).sum() != 0:
        print('error', c, d, cnt)
    print(c, d, cnt)

cat3 {'B': 'C'} [83634, 147361, 9005]
cat4 {'A': 'B', 'D': 'B'} [239397, 603]
cat6 {'D': 'A', 'E': 'B', 'G': 'C', 'H': 'B', 'I': 'A'} [234203, 5145, 652]
cat7 {'A': 'B', 'C': 'B', 'F': 'D', 'I': 'B'} [4606, 19784, 214027, 1583]
cat8 {'B': 'G', 'F': 'E'} [30338, 96743, 2953, 76085, 33881]
cat9 {'C': 'H', 'D': 'B', 'E': 'L'} [10678, 2846, 85944, 8320, 19987, 40070, 5501, 16743, 33793, 7819, 3331, 4968]


In [6]:
df_train.head()

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target,target A
id,,,,,,,,,,,,,,,,,,,,,
267387,A,B,B,C,B,B,A,E,G,G,...,0.69220,0.30912,0.47134,0.53290,0.97139,0.59414,0.36856,0.42951,8.13741,False
410037,A,A,A,A,B,B,A,E,C,F,...,0.97199,0.62596,0.40193,0.49181,0.56479,0.39386,0.36247,0.71541,7.40427,True
139373,A,A,B,A,B,B,A,E,C,J,...,0.41310,0.27010,0.40112,0.53052,0.35635,0.33341,0.33354,0.82542,7.98022,False
113765,A,A,A,C,B,B,A,E,C,G,...,0.42301,0.28331,0.46425,0.38302,0.38054,0.68827,0.34895,0.73375,7.26085,True
179915,A,B,A,C,B,B,A,E,E,L,...,0.37466,0.46316,0.75578,0.55676,0.31308,0.43157,0.32302,0.18028,7.26448,True


In [7]:
df_train.columns

Index(['cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8',
       'cat9', 'cont0', 'cont1', 'cont2', 'cont3', 'cont4', 'cont5', 'cont6',
       'cont7', 'cont8', 'cont9', 'cont10', 'cont11', 'cont12', 'cont13',
       'target', 'target A'],
      dtype='object')

In [8]:
import warnings
with warnings.catch_warnings(): 
    warnings.simplefilter("ignore")
    from sklearn.compose import ColumnTransformer
    from sklearn.pipeline import make_pipeline
    from sklearn.model_selection import cross_val_score, KFold
    from sklearn.preprocessing import StandardScaler, OneHotEncoder
    from sklearn.decomposition import PCA
cv = KFold(5, random_state=123, shuffle=True)

In [9]:
# 2번 문제에서 가져온 다중공선성있는 요소들
X_vif = ['cont0', 'cont5', 'cont8', 'cont9', 'cont12']
df_train[X_vif].corr()

,cont0,cont5,cont8,cont9,cont12
cont0,1.000000,0.573894,0.579618,0.524481,0.473249
cont5,0.573894,1.000000,0.606329,0.615449,0.631540
cont8,0.579618,0.606329,1.000000,0.558560,0.530215
cont9,0.524481,0.615449,0.558560,1.000000,0.538278
cont12,0.473249,0.631540,0.530215,0.538278,1.000000


In [10]:
X_input = df_train.columns[:-2]
X_input

Index(['cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8',
       'cat9', 'cont0', 'cont1', 'cont2', 'cont3', 'cont4', 'cont5', 'cont6',
       'cont7', 'cont8', 'cont9', 'cont10', 'cont11', 'cont12', 'cont13'],
      dtype='object')

In [11]:
[i for i in df_train.columns if (i.startswith('cont')) & (i not in X_vif)]

['cont1',
 'cont2',
 'cont3',
 'cont4',
 'cont6',
 'cont7',
 'cont10',
 'cont11',
 'cont13']

In [109]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error as mse

ct = ColumnTransformer([
    ('pca', PCA(n_components=0.8, random_state=123), X_vif),
    ('std', StandardScaler(), [i for i in df_train.columns if (i.startswith('cont')) & (i not in X_vif)] ),
    ('ohe', OneHotEncoder(drop="first"), [i for i in df_train.columns if i.startswith('cat')]),
])
reg_l2 = make_pipeline(ct, Ridge(0.1))
scores_ = cross_val_score(reg_l2, X=df_train[X_input], y=df_train['target'], cv=cv, scoring="neg_mean_squared_error")
scores_, np.mean(scores_)

(array([-0.74990495, -0.74968553, -0.75681724, -0.74735325, -0.74242976]),
 -0.7492381471930587)

## test 셋과 정답셋

In [66]:
df_test = pd.read_csv("test_prob.csv", index_col="id")
df_test.head()

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
id,,,,,,,,,,,,,,,,,,,,,
464589,A,A,A,A,B,B,A,E,A,H,...,0.28209,0.39187,0.37791,0.35714,0.43700,0.32367,0.54198,0.37136,0.38006,0.73431
39160,A,A,A,C,B,D,A,E,C,F,...,0.27939,0.40697,0.72917,0.68887,0.46054,0.58760,0.91763,0.85978,0.38625,0.38306
55733,A,A,A,A,B,D,A,E,C,F,...,0.27850,0.56523,0.71266,0.88031,0.45310,0.59965,0.80885,0.60043,0.86587,0.71623
97702,A,A,A,C,B,D,A,E,C,K,...,0.61301,0.93478,0.37334,0.94991,0.97579,0.83216,0.74999,0.81728,0.61453,0.81351
337310,A,B,A,C,B,D,A,E,C,H,...,0.72733,0.68813,0.45907,0.69450,0.44414,0.45540,0.41404,0.47312,0.70891,0.35307


In [15]:
df_ans = pd.read_csv("test_prob_ans.csv", index_col="id")
df_ans.head()

,target
id,
464589,7.029859
39160,7.192453
55733,5.785315
97702,8.137591
337310,7.927125


## test 셋의 카테고리가 train과 일치하는지 판별하기

In [67]:
# 결측치 확인
df_test.isna().sum().sum()

0

In [68]:
# cat1에 대해서 확인, set으로 하면 순서 상관없이 요소 유일하게 비교
set(df_test["cat1"].unique()) != set(df_train["cat1"].unique())

False

In [75]:
cat_diff = []
for i in range(10):
    cat = "cat{}".format(i)
    check = set(df_test[cat].unique()) != set(df_train[cat].unique())
    if check:
        cat_diff = cat_diff + [cat]
cat_diff

['cat3', 'cat4', 'cat6', 'cat7', 'cat8', 'cat9']

In [77]:
for i in cat_diff:
    print(df_test[i].unique(), df_train[i].unique())
    print(df_test[i].value_counts())

['A' 'C' 'D' 'B'] ['C' 'A' 'D']
C    36892
A    20830
D     2169
B      109
Name: cat3, dtype: int64
['B' 'C' 'D' 'A'] ['B' 'C']
B    59490
A      211
C      164
D      135
Name: cat4, dtype: int64
['A' 'C' 'B' 'I' 'H' 'D' 'E' 'G'] ['A' 'B' 'C']
A    58580
B     1221
C      159
D       29
E        5
H        3
I        2
G        1
Name: cat6, dtype: int64
['E' 'D' 'B' 'G' 'F' 'C' 'A'] ['E' 'D' 'G' 'B']
E    53604
D     4798
B     1162
G      378
F       53
A        4
C        1
Name: cat7, dtype: int64
['A' 'C' 'G' 'E' 'D' 'F' 'B'] ['G' 'C' 'E' 'A' 'D']
C    24311
E    18976
G     8326
A     7540
D      741
F      104
B        2
Name: cat8, dtype: int64
['H' 'F' 'K' 'J' 'N' 'O' 'A' 'L' 'G' 'I' 'M' 'B' 'C' 'D' 'E'] ['G' 'F' 'J' 'L' 'K' 'I' 'H' 'B' 'O' 'A' 'N' 'M']
F    21337
I     9994
L     8466
H     4936
K     4212
A     2730
G     2089
M     2019
J     1480
O     1205
N      781
B      672
C       45
D       20
E       14
Name: cat9, dtype: int64


In [94]:
# 작업결과 수작업 코드로 체크하기 위해서 확인
ret = []

cat3_ret = df_test["cat3"].value_counts().sort_index()
cat3_ret["C"] = cat3_ret["C"] + cat3_ret["B"]
cat3_ret = cat3_ret.drop(index="B")
ret = ret + [list(cat3_ret.values)]

cat4_ret = df_test["cat4"].value_counts().sort_index()
cat4_ret["B"] = cat4_ret["B"] + cat4_ret["A"] + cat4_ret["D"]
cat4_ret = cat4_ret.drop(index=["A", "D"])
ret = ret + [list(cat4_ret.values)]

cat6_ret = df_test["cat6"].value_counts().sort_index()
cat6_ret["A"] = cat6_ret["A"] + cat6_ret["D"] + cat6_ret["I"]
cat6_ret["B"] = cat6_ret["B"] + cat6_ret["E"] + cat6_ret["H"]
cat6_ret["C"] = cat6_ret["C"] + cat6_ret["G"]
cat6_ret = cat6_ret.drop(index=["D", "E", "G", "H", "I"])
ret = ret + [list(cat6_ret.values)]

# cat7에 "I" 품목이 test셋엔 없어서 생략
cat7_ret = df_test["cat7"].value_counts().sort_index()
cat7_ret["B"] = cat7_ret["B"] + cat7_ret["A"] + cat7_ret["C"]
cat7_ret["D"] = cat7_ret["D"] + cat7_ret["F"]
cat7_ret = cat7_ret.drop(index=["A", "C", "F"])
ret = ret + [list(cat7_ret.values)]

cat8_ret = df_test["cat8"].value_counts().sort_index()
cat8_ret["G"] = cat8_ret["G"] + cat8_ret["B"]
cat8_ret["E"] = cat8_ret["E"] + cat8_ret["F"]
cat8_ret = cat8_ret.drop(index=["B", "F"])
ret = ret + [list(cat8_ret.values)]

cat9_ret = df_test["cat9"].value_counts().sort_index()
cat9_ret["B"] = cat9_ret["B"] + cat9_ret["D"]
cat9_ret["H"] = cat9_ret["H"] + cat9_ret["C"]
cat9_ret["L"] = cat9_ret["L"] + cat9_ret["E"]
cat9_ret = cat9_ret.drop(index=["D", "C", "E"])
ret = ret + [list(cat9_ret.values)]

ret

[[20830, 37001, 2169],
 [59836, 164],
 [58611, 1229, 160],
 [1167, 4851, 53604, 378],
 [7540, 24311, 741, 19080, 8328],
 [2730, 692, 21337, 2089, 4981, 9994, 1480, 4212, 8480, 2019, 781, 1205]]

In [80]:
.values

array([36892, 20830,  2169,   109])

In [95]:
# train 전처리 적용
# cat7에 "I" 품목이 test셋엔 없어서 생략
repl_list = [
    ('cat3', {'B': 'C'}, [20830, 37001, 2169]),
    ('cat4', {'A': 'B', 'D': 'B'}, [59836, 164]), 
    ('cat6', {'D': 'A', 'E': 'B', 'G': 'C', 'H': 'B', 'I': 'A'}, [58611, 1229, 160]),
    ('cat7', {'A': 'B', 'C': 'B', 'F': 'D'}, [1167, 4851, 53604, 378]),
    ('cat8', {'B': 'G', 'F': 'E'}, [7540, 24311, 741, 19080, 8328]),
    ('cat9', {'C': 'H', 'D': 'B', 'E': 'L'}, [2730, 692, 21337, 2089, 4981, 9994, 1480, 4212, 8480, 2019, 781, 1205])
]

for c, d, cnt in repl_list:
    # 위에서 설계한 로직을 적용합니다.
    # 처리한 값을 저장합니다.
    df_test[c] = df_test[c].replace(d)
    # 카운트가 다른 게 있는지 확인 합니다.
    if (df_test[c].value_counts().sort_index() != cnt).sum() != 0:
        print('error', c, d, cnt)
    print(c, d, cnt)

cat3 {'B': 'C'} [20830, 37001, 2169]
cat4 {'A': 'B', 'D': 'B'} [59836, 164]
cat6 {'D': 'A', 'E': 'B', 'G': 'C', 'H': 'B', 'I': 'A'} [58611, 1229, 160]
cat7 {'A': 'B', 'C': 'B', 'F': 'D'} [1167, 4851, 53604, 378]
cat8 {'B': 'G', 'F': 'E'} [7540, 24311, 741, 19080, 8328]
cat9 {'C': 'H', 'D': 'B', 'E': 'L'} [2730, 692, 21337, 2089, 4981, 9994, 1480, 4212, 8480, 2019, 781, 1205]


In [96]:
# 트레이닝셋과 차이 있는지 체크
cat_diff_2 = []
for i in range(10):
    cat = "cat{}".format(i)
    check = set(df_test[cat].unique()) != set(df_train[cat].unique())
    if check:
        cat_diff_2 = cat_diff_2 + [cat]
cat_diff_2

[]

## test 셋으로 점수 내기

In [106]:
X_input

Index(['cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8',
       'cat9', 'cont0', 'cont1', 'cont2', 'cont3', 'cont4', 'cont5', 'cont6',
       'cont7', 'cont8', 'cont9', 'cont10', 'cont11', 'cont12', 'cont13'],
      dtype='object')

In [120]:
# 기본점수 확보
# reg_l2 fitting 시키고 test셋 예측
reg_l2.fit(X=df_train[X_input], y=df_train['target'])
pd.DataFrame(
    reg_l2.predict(df_test[X_input]),
    index=df_test.index,
    columns=['target']
).to_csv('answer6.csv')

In [122]:
# 이때의 오차 기록
mse(y_true=df_ans["target"], y_pred=reg_l2.predict(df_test[X_input]))

0.7534794238467234

L2 썼을 때 에러기록  
(array([-0.74990495, -0.74968553, -0.75681724, -0.74735325, -0.74242976]), -0.7492381471930587)  
테스트데이터셋 에러 0.7534794238467234  

### 다른 회귀모델 서치

In [130]:
from sklearn.linear_model import LinearRegression

reg_lr = make_pipeline(ct, LinearRegression())
scores_ = cross_val_score(reg_lr, X=df_train[X_input], y=df_train['target'], cv=cv, scoring="neg_mean_squared_error")
print("[LR] 학습 데이터셋에서 에러", scores_, np.mean(scores_))

[LR] 학습 데이터셋에서 성능 [-0.74990499 -0.74968555 -0.75681736 -0.74735318 -0.74242974] -0.7492381624593665


In [131]:
reg_lr.fit(X=df_train[X_input], y=df_train['target'])
print("[LR] 테스트데이터셋에서 에러", mse(y_true=df_ans["target"], y_pred=reg_lr.predict(df_test[X_input])))

[LR] 테스트데이터셋에서 성능 0.7534793729981695


In [132]:
from sklearn.linear_model import Lasso

reg_l1 = make_pipeline(ct, Lasso(0.1))
scores_ = cross_val_score(reg_l1, X=df_train[X_input], y=df_train['target'], cv=cv, scoring="neg_mean_squared_error")
print("[Lasso] 학습 데이터셋에서 에러", scores_, np.mean(scores_))

[Lasso] 학습 데이터셋에서 성능 [-0.7866382  -0.7865512  -0.7909526  -0.78725091 -0.77993245] -0.7862650737254838


In [133]:
reg_l1.fit(X=df_train[X_input], y=df_train['target'])
print("[Lasso] 테스트데이터셋에서 에러", mse(y_true=df_ans["target"], y_pred=reg_l1.predict(df_test[X_input])))

[Lasso] 테스트데이터셋에서 성능 0.7914154849133085


In [137]:
from sklearn.ensemble import BaggingRegressor

reg_bagg = make_pipeline(ct, BaggingRegressor(
    n_estimators=20, random_state=123
))
scores_ = cross_val_score(reg_bagg, df_train[X_input], df_train['target'], cv=cv, scoring="neg_mean_squared_error")
print("[Bagging] 학습 데이터셋에서 에러", scores_, np.mean(scores_))

[Bagging] 학습 데이터셋에서 에러 [-0.78497516 -0.78512242 -0.79132555 -0.78424848 -0.77549323] -0.7842329683065039


In [138]:
reg_bagg.fit(X=df_train[X_input], y=df_train['target'])
print("[Bagging] 테스트데이터셋에서 에러", mse(y_true=df_ans["target"], y_pred=reg_bagg.predict(df_test[X_input])))

[Bagging] 테스트데이터셋에서 에러 0.7845141476903746


In [139]:
from sklearn.ensemble import RandomForestRegressor

reg_rf = make_pipeline(ct, RandomForestRegressor(
    n_estimators=10, max_depth=3, min_samples_split= 100, random_state=123
))
scores_ = cross_val_score(reg_rf, df_train[X_input], df_train['target'], cv=cv, scoring="neg_mean_squared_error")
print("[RandomForest] 학습 데이터셋에서 에러", scores_, np.mean(scores_))

[RandomForest] 학습 데이터셋에서 에러 [-0.7624887  -0.7624305  -0.76737774 -0.76233356 -0.75654932] -0.7622359639782273


In [140]:
reg_rf.fit(X=df_train[X_input], y=df_train['target'])
print("[RandomForest] 테스트데이터셋에서 에러", mse(y_true=df_ans["target"], y_pred=reg_rf.predict(df_test[X_input])))

[RandomForest] 테스트데이터셋에서 에러 0.7687200472510402


In [143]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor

reg_adb = make_pipeline(ct, AdaBoostRegressor(
    base_estimator=DecisionTreeRegressor(max_depth=2, min_samples_leaf=100), n_estimators=3, learning_rate=0.5, random_state=123
))
scores_ = cross_val_score(reg_adb, df_train[X_input], df_train['target'], cv=cv, scoring="neg_mean_squared_error")
print("[AdaBoost] 학습 데이터셋에서 에러", scores_, np.mean(scores_))

[AdaBoost] 학습 데이터셋에서 에러 [-0.76782987 -0.76752361 -0.7722455  -0.7663888  -0.76114971] -0.7670274985678095


In [144]:
reg_l1.fit(X=df_train[X_input], y=df_train['target'])
print("[AdaBoost] 테스트데이터셋에서 에러", mse(y_true=df_ans["target"], y_pred=reg_l1.predict(df_test[X_input])))

[AdaBoost] 테스트데이터셋에서 에러 0.7914154849133085


In [145]:
from xgboost import XGBRegressor

reg_xgb = make_pipeline(ct, XGBRegressor(
    colsample_bytree=0.25, n_estimators=500, max_depth=2, random_state=123
))
scores_ = cross_val_score(reg_xgb, df_train[X_input], df_train['target'], cv=cv, scoring="neg_mean_squared_error")
print("[xgboost] 학습 데이터셋에서 에러", scores_, np.mean(scores_))

[xgboost] 학습 데이터셋에서 에러 [-0.7315781  -0.73241635 -0.73906267 -0.72923725 -0.72423418] -0.731305709180708


In [146]:
reg_xgb.fit(X=df_train[X_input], y=df_train['target'])
print("[xgboost] 테스트데이터셋에서 에러", mse(y_true=df_ans["target"], y_pred=reg_xgb.predict(df_test[X_input])))

[xgboost] 테스트데이터셋에서 에러 0.7364352954406531


In [147]:
from sklearn.ensemble import VotingRegressor

# 앙상블시 가장 좋은 성능을 보인 조합입니다.
reg_vt = VotingRegressor([
        ('lr', reg_lr), # LinearRegression
        ('lr_2', reg_l2), # Ridge
        ('rf', reg_rf), # RandomForest
        ('xgb', reg_xgb) # xgboost
])
scores_ = cross_val_score(reg_vt, df_train[X_input], df_train['target'], cv=cv, scoring="neg_mean_squared_error")
print("[Voting] 학습 데이터셋에서 에러", scores_, np.mean(scores_))

[Voting] 학습 데이터셋에서 에러 [-0.74271021 -0.74270667 -0.74887949 -0.74055198 -0.73526136] -0.7420219429763866


In [148]:
reg_vt.fit(X=df_train[X_input], y=df_train['target'])
print("[Voting] 테스트데이터셋에서 에러", mse(y_true=df_ans["target"], y_pred=reg_vt.predict(df_test[X_input])))

[Voting] 테스트데이터셋에서 에러 0.7469883924033374


In [149]:
# Voting 모델로 결과 제출
pd.DataFrame(
    reg_vt.predict(df_test[X_input]),
    index=df_test.index,
    columns=['target']
).to_csv('answer6.csv')

In [150]:
# 문제 4에서 뽑아낸 파생변수를 만들어봅니다.

q = [i * 0.01 for i in range(101)]
for i in range(0, 14):
    col = 'cont{}'.format(i)
    col_q = col + '_q' 
    q_val = df_train[col].quantile(q)
    q_val.iloc[[0, -1]] = [-np.inf, np.inf]
    q_cat = pd.cut(df_train[col], bins=q_val, right=True)
    q_mean = df_train.groupby(q_cat)['target'].mean()
    df_train[col_q] = q_cat.map(q_mean).astype('float')
    df_test[col_q] = pd.cut(df_test[col], bins=q_val, right=True).map(q_mean).astype('float')

In [159]:
# df_train.columns[-14:]
X_input_2 = [i for i in df_train.columns if i.startswith('cat')] + [i for i in df_train.columns[-14:]]

In [172]:
# 파생변수 적용한 Ridge
ct_2 = ColumnTransformer([
    ('pt', "passthrough", [i for i in df_train.columns[-14:]]),
    ('ohe', OneHotEncoder(drop="first"), [i for i in df_train.columns if i.startswith('cat')]),
])
reg_l2_2 = make_pipeline(ct_2, Ridge(0.2))
scores_ = cross_val_score(reg_l2_2, df_train[X_input_2], df_train['target'], cv=cv, scoring="neg_mean_squared_error")
print("[Ridge_2] 학습 데이터셋에서 에러", scores_, np.mean(scores_))
reg_l2_2.fit(X=df_train[X_input_2], y=df_train['target'])
print("[Ridge_2] 테스트데이터셋에서 에러", mse(y_true=df_ans["target"], y_pred=reg_l2_2.predict(df_test[X_input_2])))

[Ridge_2] 학습 데이터셋에서 에러 [-0.71515745 -0.71661105 -0.72193934 -0.71221067 -0.70710866] -0.714605433545979
[Ridge_2] 테스트데이터셋에서 에러 0.7238492549712147


In [162]:
reg_lr_2 = make_pipeline(ct_2, LinearRegression())
scores_ = cross_val_score(reg_lr_2, df_train[X_input_2], df_train['target'], cv=cv, scoring="neg_mean_squared_error")
print("[LinearRegression_2] 학습 데이터셋에서 에러", scores_, np.mean(scores_))
reg_lr_2.fit(X=df_train[X_input_2], y=df_train['target'])
print("[LinearRegression_2] 테스트데이터셋에서 에러", mse(y_true=df_ans["target"], y_pred=reg_lr_2.predict(df_test[X_input_2])))

[LinearRegression_2] 학습 데이터셋에서 에러 [-0.7151571  -0.71661149 -0.7219398  -0.71221046 -0.70710867] -0.7146055045592127
[LinearRegression_2] 테스트데이터셋에서 에러 0.7238503275095004


In [ ]:
reg_rf_2 = make_pipeline(ct_2, RandomForestRegressor(
    n_estimators=10, max_depth=3, min_samples_split= 100, random_state=123
))
scores_ = cross_val_score(reg_rf_2, df_train[X_input_2], df_train['target'], cv=cv, scoring="neg_mean_squared_error")
print("[RandomForest] 학습 데이터셋에서 에러", scores_, np.mean(scores_))

In [ ]:
reg_rf_2.fit(X=df_train[X_input_2], y=df_train['target'])
print("[RandomForest] 테스트데이터셋에서 에러", mse(y_true=df_ans["target"], y_pred=reg_rf_2.predict(df_test[X_input_2])))

In [ ]:
### 그리드 서치로 하이퍼파라미터 탐색

In [168]:
from sklearn.model_selection import GridSearchCV

# RandomForest의 최적 하이퍼파라미터 그리드서치
gscv = GridSearchCV(
    estimator=RandomForestRegressor(random_state=123), # Hyper Parameter 탐색할 모델을 세팅합니다. 
    param_grid={
        'n_estimators': [5, 10, 15],
        'max_depth': [3, 5, 7],
        'min_samples_split': [128, 256, 512]
    }, # dict 형식으로 param_grid를 정의합니다. key: 대상 Hyper Parameter, value: 후보 설정값 (List)
    scoring='neg_mean_squared_error', # _score로 끝나면 _score를 빼고 넘기고, _error끝나면 neg_를 앞에 붙입니다, ex: neg_mean_squared_error,
    cv = cv, # 5겹 교차검증을 정의합니다. 
    iid = False # 단순평균을 사용합니다. True면 겹외셋의 수에 따라 가중평균으로 구합니다.
)
do = make_pipeline(ct_2, gscv)
do.fit(df_train[X_input_2], df_train['target'])
# cv_results_에는 상세 결과가 들어가 있습니다.
best_param, best_score = gscv.best_params_, gscv.best_score_
best_param, best_score

({'max_depth': 7, 'min_samples_split': 128, 'n_estimators': 15},
 -0.7368605671495316)

In [169]:
# xgboost의 최적 하이퍼파라미터 그리드서치

gscv_xgb = GridSearchCV(
    estimator=XGBRegressor(random_state=123), # Hyper Parameter 탐색할 모델을 세팅합니다. 
    param_grid={
        'colsample_bytree': [0.1, 0.25, 0.5],
        'max_depth': [2, 3],
        'n_estimators': [300, 500, 700]
    }, # dict 형식으로 param_grid를 정의합니다. key: 대상 Hyper Parameter, value: 후보 설정값 (List)
    scoring='neg_mean_squared_error', # _score로 끝나면 _score를 빼고 넘기고, _error끝나면 neg_를 앞에 붙입니다, ex: neg_mean_squared_error,
    cv = cv, # 5겹 교차검증을 정의합니다. 
    iid = False # 단순평균을 사용합니다. True면 겹외셋의 수에 따라 가중평균으로 구합니다.
)
do_xgb = make_pipeline(ct_2, gscv_xgb)
do_xgb.fit(df_train[X_input_2], df_train['target'])
# cv_results_에는 상세 결과가 들어가 있습니다.
best_param, best_score = gscv_xgb.best_params_, gscv_xgb.best_score_
best_param, best_score

({'colsample_bytree': 0.1, 'max_depth': 3, 'n_estimators': 700},
 -0.7129269595085164)

In [173]:
reg_rf_best = make_pipeline(ct_2, RandomForestRegressor(
    **{'max_depth': 7, 'min_samples_split': 128, 'n_estimators': 15}, random_state=123
))
reg_xgb_best = make_pipeline(ct_2, XGBRegressor(
    **{'colsample_bytree': 0.1, 'max_depth': 3, 'n_estimators': 700}, random_state=123
))


# 앙상블시 가장 좋은 성능을 보인 조합들 선택
reg_vt_best = VotingRegressor([
        ('lr_2', reg_l2_2), # Ridge
        ('rf', reg_rf_best), # RandomForest
        ('xgb', reg_xgb_best) # xgboost
])
scores_ = cross_val_score(reg_vt_best, df_train[X_input_2], df_train['target'], cv=cv, scoring="neg_mean_squared_error")
print("[Voting2] 학습 데이터셋에서 에러", scores_, np.mean(scores_))

[Voting] 학습 데이터셋에서 에러 [-0.71666961 -0.71766588 -0.72201289 -0.71309604 -0.7080289 ] -0.7154946618184116


In [175]:
reg_vt_best.fit(X=df_train[X_input_2], y=df_train['target'])
pred_result = reg_vt_best.predict(df_test[X_input_2])
print("[Voting2] 테스트데이터셋에서 에러", mse(y_true=df_ans["target"], y_pred=pred_result))

[Voting2] 테스트데이터셋에서 에러 0.7229046023448432


In [ ]:
# 최종 Voting 모델로 결과 제출
pd.DataFrame(
    pred_result,
    index=df_test.index,
    columns=['target']
).to_csv('answer6.csv')